# [11] Segmentation 손실 함수 만들기

`U-Net`의 학습을 위한 손실 함수를 만들어보겠습니다.

자료는 https://blog.naver.com/PostView.nhn?blogId=sogangori&logNo=221087066947 과 https://www.jeremyjordan.me/semantic-segmentation/ 를 재구성하여 만들었습니다.

In [1]:
import torch
import torch.nn as nn
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
import torch.nn.functional as F

Q) Segmentation Loss는 기본적으로 어떤 형태로 되어야할까요?

![Segmentation_loss](./imgs/Segmentation_loss.png)

A) pixel-wise하게 확률을 구하여서 원래는 pixel wise한 cross-entropy를 계산함.

$$ Dice = \frac{2|A \cap B|}{|A| + |B|}$$

이 방법이 가장 대중적인 방법이지만, 또 다른 대중적인 Loss가 있는데 `Dice Coefficient`이다.

Dice coefficient는 두 샘플의 overlap (겹침) 부분을 측정하는 것입니다. 값은 0에서 1사이에 존재합니다.

Dice loss는 아래처럼 구함.

![dice_coefficient](./imgs/dice_coefficient.png)

![dice](./imgs/dice_loss.png)

BCE Loss 의 구체적 설명은 https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html?highlight=bce#torch.nn.BCELoss 을 참고하세요.

In [ ]:
class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        BCE = nn.BCELoss()(inputs, targets)
        num = targets.size(0)     
        inputs = inputs.reshape(num, -1)
        targets = targets.reshape(num, -1)
        
        intersection = ????????
        dice = ????????
        dice_loss = 1 - dice.sum()/num

        loss_final = 3*BCE + dice_loss
        return loss_final

In [ ]:
class DiceScore(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceScore, self).__init__()

    def forward(self, inputs, targets, smooth=1):
              
        
        intersection = ????????                   
        dice_score = ????????
        return dice_score

## Focal Loss

Focal Loss는 object detection에서 주로 많이 사용된다.

Focal loss는 클래스간 불균형이 극도로 심한 상황 (예: 1:1000)을 해결하기 위해 디자인 되었다.

요약하면, 성능이 좋은 class에 대해서는 loss를 적게 줘서 loss 갱신을 거의 하지 못하게하고, 그렇지 않은 class에 대해서는 loss를 크게 줘서 loss 갱신을 크게 하는 것입니다. (CE: cross entropy loss)

$$ FL(p_t) = - (1-p_t)^{\gamma} CE $$

이 focal loss의 $\gamma$를 0~5 사이의 값으로 조절했을 때 아래 그림을 얻을 수 있습니다.

![fig_focal_loss](./imgs/fig_focal_loss.png)

$\gamma$가 0이면 FL은 CE가 됩니다.

### [1]

만약 example 을 잘못 분류했는데, $p_t$ 가 작은 값인 경우 조절항이 1에 가까워지면서 조잘 항의 값이 거의 0이 되므로 결국 로스 함수에 영향을 주지 않는다.

$$((1-0.00001)^{\gamma}= \sim 1)$$

$p_t \rightarrow 1$ 에 가까워 지면, 조절 항은 거의 0 이 된다. 

$$((1-0.999)^{\gamma}= \sim 0)$$

이것이 잘 분류된 샘플을 down-weight 시키는 경우이다.

### [2] 

focusing parameter $\gamma$ 는 부드럽게 easy example을 down-weight 시킨다.

$\gamma$ 의 값이 커질수록 FL 의 down-weight 시키는 영향력이 커진다.(논문에서는 $\gamma=2$인경우 가장 성능이 좋았다)

직관적으로 보면, modulating factor 은 easy example들의 loss 에 대한 영향력을 줄이며, loss 를 낮추는 example 의 범위를 결정한다.

예를 들어 $\gamma=2$ 인 경우 $p_t = 0.9$ 로 예측되었던 example 의 원래 CE 로스에 비해서 FL 에서는 CE의 $p_t=\sim 0.968$ 일때 수준의 작은 loss를 받게 되므로 상대적으로 1000x 작아 진다.

이것은 잘못  분류되었던 examples 의 중요도를 높이는 역활을 한다.

($\gamma=2$ 로 셋팅되었을 때, $p_t \leq 0.5$ 로 예측된 example 의 loss 는 단지 4x 작아질 뿐이다.)


In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = 1
        self.gamma = 2
        
    def forward(self, inputs, targets):
        num = targets.size(0)
        inputs = inputs.reshape(num, -1)
        targets = targets.reshape(num, -1)
        BCE_loss = ????????
        batch_loss = ????????
        loss = batch_loss.mean()
        
        return loss